# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра». В нашем распоряжении данные о поведении клиентов, которые уже перешли на тарифы Smart и Ultra. 

**Цель исследования:**
построить модель для задачи классификации, которая выберет подходящий тариф для конкретного пользователя.

**Ход исследования:**

1. Загрузка данных и изучение первичной информации.
2. Подготовка данных к обучению моделей.
3. Исследование качества разных моделей.
4. Итоговое тестирование выбранной модели.

**Описание данных:**

- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Описание тарифов:**
1. Тариф «Смарт»:
 - Ежемесячная плата: 550 рублей
 - Включено 500 минут разговора, 50 сообщений и 15 Гб интернет-трафика
 - Стоимость услуг сверх тарифного пакета:
 - Минута разговора — 3 рубля. Количество использованных минут и мегабайтов «Мегалайн» всегда округляет вверх. Если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута.
 - Сообщение — 3 рубля.
 - 1 Гб интернет-трафика — 200 рублей.
2. Тариф «Ультра»
 - Ежемесячная плата: 1950 рублей
 - Включено 3000 минут разговора, 1000 сообщений и 30 Гб интернет-трафика
 - Стоимость услуг сверх тарифного пакета:
 - Минута разговора — 1 рубль;
 - Сообщение — 1 рубль;
 - 1 Гб интернет-трафика: 150 рублей.

## 1. Открытие файла с данными и изучение общей информации

Загрузим необходимые библиотеки

In [1]:
from sklearn import tree

from sklearn.dummy import DummyClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

import pandas as pd

import matplotlib.pyplot as plt


In [2]:
RANDOM_STATE = 0    #константа для обучения моделей

Сохраним наш файл с данными в переменную data. Выведем любые 10 строк исходных данных, а также просмотрим основную информацию о наборе данных.

In [3]:
data = pd.read_csv('users_behavior.csv')
display(data.sample(10))
data.info()

,calls,minutes,messages,mb_used,is_ultra
321,38.0,237.78,5.0,4905.50,0
1054,42.0,226.18,21.0,13243.48,0
1132,51.0,381.12,43.0,14771.91,0
2663,60.0,449.57,0.0,20700.96,0
3111,64.0,513.57,21.0,16861.47,0
1524,41.0,215.34,7.0,10842.36,0
2454,30.0,225.11,30.0,12280.81,0
3065,91.0,610.31,46.0,23953.48,0
2246,89.0,669.06,18.0,3139.82,1
631,69.0,389.89,103.0,22633.64,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Проверим данные на пропуски

In [4]:
display(pd.DataFrame(data.isna().mean()*100).style.background_gradient('coolwarm'))

,0
calls,0.000000
minutes,0.000000
messages,0.000000
mb_used,0.000000
is_ultra,0.000000


Пропусков не нашли. Теперь проверим данные на дубликаты

In [5]:
print('Количество дубликатов:', data.duplicated().sum())

Количество дубликатов: 0


Произведем проверку набора данных на сбалансированность классов в признаке, определяющем клиентский тариф.

In [6]:
data['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Заметен дисбаланс классов в сторону использования клиентами тарифа "Смарт". Эта информация нам пригодится при обучении моделей предсказания тарифа. Посмотрим на корреляцию признаков в данных

In [7]:
data.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


Видим, что присутствует примерно одинаковая степень зависимости между каждым обычным и целевым признаком. При этом заметна мультиколлениарность между признаками calls и minutes.

**Итог.**
В ходе загрузки данных и изучение общей информации было сделано следующее:
1. проверили данные на пропуски и дуюликаты;
2. выясняли, что присутствует дисбаланс классов в признаке, определяющем клиентский тариф;
3. проанализировали информацию о корреляции признаков в наборе данных.

## 2. Подготовка данных к обучению моделей

Проведем разделение исходного набора данных на 2 выборки: обучающую и тестовую (сделаем это в соотношении 4:1).

In [8]:
train, test = train_test_split(data, train_size=0.8, random_state=RANDOM_STATE, stratify=data['is_ultra'])

In [9]:
print(train.shape, test.shape)

(2571, 5) (643, 5)


Выделим из каждой выборки фичи и таргет

In [10]:
X_train = train.drop(['is_ultra'], axis=1)
Y_train = train['is_ultra']
X_test = test.drop(['is_ultra'], axis=1)
Y_test = test['is_ultra']

**Итог.**
Получили тренировочный и тестовый набор данных. Каждый из них был разделен на подвыборки с обычными и целевыми признаками. В дальнейшем с помощью обучающей выборки будет происходить обучение модели, а также подбор гиперпараметров, а с помощью тестовой выборки проверим качество выбранной модели.

## 3. Исследование качества разных моделей

Начнем исследование качества моделей с решающего дерева. Т.к. мы имеем дисбаланс классов в целевом признаке, то сразу укажем такой гиперпараметр, как class_weight, который выставит каждому классу необходимы вес. 

In [11]:
model = DecisionTreeClassifier(random_state=RANDOM_STATE, class_weight='balanced')
grid_space = {
    'max_depth':range(1, 10, 1)
}
grid = GridSearchCV(model, param_grid=grid_space, cv=5, scoring='accuracy')
model_grid = grid.fit(X_train, Y_train)

In [12]:
print('Наилучшие гиперпараметры: '+str(model_grid.best_params_))
print('Значение метрики accuracy: '+str(model_grid.best_score_))

Наилучшие гиперпараметры: {'max_depth': 3}
Значение метрики accuracy: 0.7868588266404745


Лучший показатель имеет решающее дерево с глубиной, равной 3. Теперь посмотрим на модель случайного леса

In [13]:
model = RandomForestClassifier(random_state=RANDOM_STATE, class_weight='balanced')
grid_space = {
    'max_depth':range(1, 15, 1),
    'n_estimators':range(1, 20, 1)
}
grid = GridSearchCV(model, param_grid=grid_space, cv=5, scoring='accuracy')
model_grid = grid.fit(X_train, Y_train)

In [14]:
print('Наилучшие гиперпараметры: '+str(model_grid.best_params_))
print('Значение метрики accuracy: '+str(model_grid.best_score_))

Наилучшие гиперпараметры: {'max_depth': 7, 'n_estimators': 7}
Значение метрики accuracy: 0.7969732915265763


Как видим, случайный лес с количеством деревьев, равным 7 и при глубине каждого в 7, показал лучшее значение accuracy, чем решающее дерево. Для итого тестирования возьмем именно эту модель с описанными гиперпараметрами.

**Итог.**
Рассмотрели 2 модели, которые не чувствительны к мультиколлениарности, описанной в пункте 1. Точность правильных ответов у дерева решений 78.7% против 79.7% у случайного леса. Именно последнюю модель и возьмем для финального тестирования.

## 4. Проверка качества модели на тестовой выборке.

Проведем проверку выбранной модели для предсказания на тестовой выборке, которую мы определили в пункте 2.

In [15]:
model = RandomForestClassifier(random_state=RANDOM_STATE, max_depth=7, n_estimators=7, class_weight='balanced')
model.fit(X_train, Y_train)
prediction = model.predict(X_test)
result = accuracy_score(Y_test, prediction)
print('Значение метрики accuracy на тестовой выборке:', result)

Значение метрики accuracy на тестовой выборке: 0.7978227060653188


Как видим из результатов метрики accuracy доля правильных ответов составляет почти 80%. Данный показатель подтверждает вменяемость выбранной модели при дисбалансе классов в 69% в сторону класса "0"

## 5. Вывод

В ходе исследования мы загрузили данные и исследовали первичную информацию о них. Исходный датасет разбили на тренировочную и тестовую выборку для обучения моделей, предсказывающих подходящий тариф клиенту. Было рассмотрено две модели: решающее дерево и случайный лес. По тестированию лучше всего показала себя модель случайного леса с количеством деревьев 7 и глубиной каждого дерева в 7. При проверке на тестовой выборке данная модель показала долю правильных предсказаний, почти равную 80%.